In [ ]:
import re
import nltk
import copy
import scipy
import tensorflow
import numpy as np 
import pandas as pd 
from tqdm.auto import tqdm 
from bs4 import BeautifulSoup 
from scipy.sparse import vstack
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.pipeline import Pipeline
from tensorflow.keras import activations
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow. keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, Bidirectional, TimeDistributed, Dense, Conv1D, MaxPool1D, concatenate, \
    Dropout, add, InputSpec, PReLU, Flatten, multiply, Reshape, Permute, BatchNormalization,LeakyReLU

# Data Load

In [ ]:
data_train=pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
data_val=pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
data_train.head()

# Give Weight to different toxicity level

In [ ]:
weight_toxic={'toxic':1.2,'severe_toxic':1.3,'obscene':0.5,'threat':0.4,'insult':0.3,'identity_hate':0.5}
data=copy.deepcopy(data_train)
data['weight_rate']=0
for col,val in weight_toxic.items():
    data['weight_rate']+=val*data[col]

In [ ]:
#Balance through down_scaling
data_insulting=data[data['weight_rate']>0]
data_val_0=data[data['weight_rate']==0].sample(len(data_insulting)+5000)
data_under_sampled=pd.concat([data_insulting,data_val_0])
data=data_under_sampled

In [ ]:
# data['rate']=data['toxic']+data['severe_toxic']+data['obscene']+data['threat']+data['insult']+data['identity_hate']
# data.rate.unique()
# print(data.rate.value_counts())

In [ ]:
# data['rate']=data.rate.apply(lambda x:1 if x>0 else 0)
# data.head()

# Data Cleaning

In [ ]:
#Cleaning Template copied from-
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text


def clean(data, col):
    
    data[col] = data[col].str.replace('https?://\S+|www\.\S+', ' social medium ', regex=True)      
        
    data[col] = data[col].str.lower()
    data[col] = data[col].str.replace("4", "a") 
    data[col] = data[col].str.replace("2", "l")
    data[col] = data[col].str.replace("5", "s") 
    data[col] = data[col].str.replace("1", "i") 
    data[col] = data[col].str.replace("!", "i") 
    data[col] = data[col].str.replace("|", "i", regex=False) 
    data[col] = data[col].str.replace("0", "o") 
    data[col] = data[col].str.replace("l3", "b") 
    data[col] = data[col].str.replace("7", "t") 
    data[col] = data[col].str.replace("7", "+") 
    data[col] = data[col].str.replace("8", "ate") 
    data[col] = data[col].str.replace("3", "e") 
    data[col] = data[col].str.replace("9", "g")
    data[col] = data[col].str.replace("6", "g")
    data[col] = data[col].str.replace("@", "a")
    data[col] = data[col].str.replace("$", "s", regex=False)
    data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")    
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace("centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", " sick ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
    data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
    data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ", regex=False)
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
    data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ", regex=False)
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you', regex=True)
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " is ", regex=False)
    data[col] = data[col].str.replace(r"\'ve", " have ", regex=False)
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ", regex=False)
    data[col] = data[col].str.replace(r"\'d", " would ", regex=False)
    data[col] = data[col].str.replace(r"\'ll", " will ", regex=False)
    data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=False)
    data[col] = data[col].str.replace('\s+', ' ', regex=True)
    data[col] = data[col].str.replace(r'(.)\1+', r'\1\1', regex=True) 
    data[col] = data[col].str.replace("[:|♣|'|§|♠|*|/|?|=|%|&|-|#|•|~|^|>|<|►|_]", '', regex=True)
    data[col] = data[col].str.replace(r"what's", "what is ")    
    data[col] = data[col].str.replace(r"\'ve", " have ", regex=False)
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ", regex=False)
    data[col] = data[col].str.replace(r"i'm", "i am ", regex=False)
    data[col] = data[col].str.replace(r"\'re", " are ", regex=False)
    data[col] = data[col].str.replace(r"\'d", " would ", regex=False)
    data[col] = data[col].str.replace(r"\'ll", " will ", regex=False)
    data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=False)
    data[col] = data[col].str.replace(r"\'s", " ", regex=False)

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', regex=True)
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1', regex=True)    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ', regex=True)    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   
    data[col] = data[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    tqdm.pandas()
    data[col] = data[col].progress_apply(text_cleaning)
    return data

# Clean the data

In [ ]:
data_clean = clean(data,'comment_text')      #Training data
data_val=clean(data_val,'less_toxic')        #Validation data
data_val=clean(data_val,'more_toxic')

In [ ]:
data_clean.head()

# Tf-Idf+Ridge

In [ ]:
# pipeline = Pipeline(
#     [
#         ("vect", TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5))),
#         ("clf", Ridge()),
#     ]
# )
# pipeline.fit(data_clean['comment_text'],data_clean['rate'])
# p1 = pipeline.predict(val_less_toxic['less_toxic'])
# p2 = pipeline.predict(val_more_toxic['more_toxic'])

# LSTM+Embedding Layer

In [ ]:
#To vectorize the text(we can also use keras functions for this)
def text_to_vector(df,col_name,w2v):
    w2v={}
    text=list(df[col_name])
    rate=list(df['weight_rate'])
    count=1
    if len(w2v)==0:
        for txt in text:
            for word in txt.split(' '):
                if word not in w2v:
                    w2v[word]=count
                    count+=1
    X=np.zeros((len(text),30),dtype='float32')
    Y=np.zeros((len(text),1),dtype='float32')
    for i,txt in enumerate(text):
        j=0
        Y[i][0]=rate[i]
        for word in txt.split(' '):
            if word in w2v and j<30:
                X[i][j]=w2v[word]
                j+=1
                
    return X,Y,w2v 

In [ ]:
x_train,y_train,w2v=text_to_vector(data_clean,col_name='comment_text',w2v={})
print(x_train.shape,y_train.shape,len(w2v))

In [ ]:
# def lstm_model(V1, V2, K, sequence_length):
#     inputs = Input(shape=(sequence_length,))
#     l0 = Embedding(V1, K, input_length=sequence_length,mask_zero=True)(inputs)
#     l0 = BatchNormalization()(l0)
#     l0 = LSTM(356)(l0)
#     l0 = BatchNormalization()(l0)
#     l0=Dense(20)(l0)
#     l0=LeakyReLU()(l0)
#     l0 = BatchNormalization()(l0)
#     out = Dense(V2, activation='linear')(l0)
#     model = Model(inputs=inputs, outputs=out)
#     opt=tensorflow.keras.optimizers.Adam(learning_rate=0.004)
# #     opt=tensorflow.keras.optimizers.SGD(learning_rate=0.001)
#     model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
#     return model

# model= lstm_model(len(w2v)+1, 1, 300, 30)
# model.summary()

In [ ]:
def lstm_model(V1, V2, K, sequence_length):
    inputs = Input(shape=(sequence_length,))
    l0 = Embedding(V1, K, input_length=sequence_length,mask_zero=True)(inputs)
    l0 = BatchNormalization()(l0)
    l0 = Bidirectional(LSTM(256))(l0)
    l0 = BatchNormalization()(l0)
    l0=Dense(10)(l0)
    l0=LeakyReLU()(l0)
    l0 = BatchNormalization()(l0)
    out = Dense(V2, activation='linear')(l0)
    model = Model(inputs=inputs, outputs=out)
    opt=tensorflow.keras.optimizers.Adam(learning_rate=0.004)
#     opt=tensorflow.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
    return model

model= lstm_model(len(w2v)+1, 1, 200, 30)
model.summary()

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
def text2vect(df,col):
    X=np.zeros((len(list(df[col])),30),dtype='float32')
    for i,txt in enumerate(list(df[col])):
        j=0
        for word in txt.split(' '):
            if word in w2v and j<30:
                X[i][j]=w2v[word]
                j+=1
    return X


def validation_result(df,col1,col2):
    less_toxic=text2vect(df,col1)
    more_toxic=text2vect(df,col2)
    match=0
    if len(less_toxic)==len(more_toxic):
        
        pred_less_toxic=model.predict(less_toxic)
        pred_more_toxic=model.predict(more_toxic)
        
        for i in tqdm(range(len(pred_less_toxic))):
            if pred_less_toxic[i]<=pred_more_toxic[i]:
                match+=1
        return match/len(pred_less_toxic)
    return 'length not same'
    

# Validation Number Check

In [ ]:
print(validation_result(data_val,'less_toxic','more_toxic'))

# Submission Part

In [ ]:
df_sub=pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')
df_sub=clean(df_sub,'text')

In [ ]:
score=model.predict(text2vect(df_sub,'text'))
score.shape

In [ ]:
df_sub['score']=score
df_sub['score'] = df_sub['score'].rank(method='first')
df_sub[['comment_id', 'score']].to_csv("submission.csv", index=False)